# Train Traffic Lights Agents

Utilise les fonctions de @Binetruy

- crée un network à partir d'un fichier .osm et des trajectoires de véhiculess
- ajoute un flux de voiture sur les routes
- personnalise un Environnement pour le RL
- integre l'environnement pour RLlib et execute la simulation


In [1]:
from flow.core.params import VehicleParams
from flow.core.params import NetParams, SumoCarFollowingParams
from flow.core.params import InitialConfig
from flow.core.params import EnvParams
from flow.core.params import SumoParams
from flow.controllers import RLController, IDMController
from flow.networks.IssyOSMNetwork import IssyOSMNetwork
from flow.core.params import InFlows
from collections import OrderedDict
import json
import ray
from ray.rllib.agents.registry import get_agent_class
from ray.tune import run_experiments
from ray.tune.registry import register_env
from flow.utils.registry import make_create_env
from flow.utils.rllib import FlowParamsEncoder
from flow.core.params import VehicleParams, SumoCarFollowingParams

/home/lino/anaconda3/envs/flow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/lino/anaconda3/envs/flow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/lino/anaconda3/envs/flow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/lino/anaconda3/envs/flow/lib/python3.6/site-packages

## Importation du network d'Issy

On vérifie si IssyOSMNetwork est bien importé.

In [2]:
from flow.networks.IssyOSMNetwork import ADDITIONAL_NET_PARAMS, EDGES_DISTRIBUTION

print(ADDITIONAL_NET_PARAMS)
print(EDGES_DISTRIBUTION)

{'speed_limit': 50}
['-100822066', '4794817', '4783299#0', '155558218']


## Ajoute les flux de voiture

`IDMController` : The Intelligent Driver Model is a car-following model specifying vehicle dynamics by a differential equation for acceleration $\dot{v}$.

`RLController` : a trainable autuonomous vehicle whose actions are dictated by an RL agent. 

In [3]:
vehicles = VehicleParams()
vehicles.add("human",acceleration_controller=(IDMController, {}), num_vehicles=10)
vehicles.add("rl",acceleration_controller=(RLController, {}), num_vehicles=2)

- `vehs_per_hour`: nombre de vehicule par heure, uniformément espacés. Par exemple, comme il y a $60 \times 60 = 3600$ secondes dans une heure, le parametre $\frac{3600}{5}=720$ va faire rentrer des vehicules dans le network toutes les $5$ secondes.

- `probability`: c'est la probabilité qu'un véhicule entre dans le network toutes les secondes. Par exemple, si on la fixe à $0.2$, alors chaque seconde de la simulation un véhicule aura $\frac{1}{5}$ chance d'entrer dans le network

- `period`: C'est le temps en secondes entre 2 véhicules qui sont insérés. Par exemple, le fixer à $5$ ferait rentrer des véhicules dans le network toutes les $5$ secondes (ce qui équivaut à mettre `vehs_per_hour` à $720$).

<font color='red'>
$\rightarrow$ Exactement 1 seul de ces 3 paramètres doit être configurer !
</font>

In [4]:
inflow = InFlows()

inflow.add(veh_type      = "human",
           edge          = "4794817",
           probability   = 0.3, 
           depart_speed  = 7,
           depart_lane   = "random")

inflow.add(veh_type      = "human",
           edge          = "4783299#0",
           probability   = 0.2,
           depart_speed  = 7,
           depart_lane   = "random")

inflow.add(veh_type       = "human",
           edge           = "-100822066",
           probability    = 0.25,
           depart_speed   = 7,
           depart_lane    = "random")

inflow.add(veh_type       = "rl",
           edge           = "-100822066",
           probability    = 0.05,
           depart_speed   = 7,
           depart_lane    = "random",
           color          = "blue")

inflow.add(veh_type       = "human",
           edge          = "155558218",
           probability   = 0.2,
           depart_speed  = 7,
           depart_lane   = "random")

## Personnalise un Environnement pour le RL

plus de méthodes sur : http://berkeleyflow.readthedocs.io/en/latest/

In [5]:
from flow.envs.IssyEnv import IssyEnv1

## Lance une simulation avec Training RLlib

Pour qu'un environnement puisse être entrainé, l'environnement doit être accessible via l'importation à partir de flow.envs. 


<font color='red'>
Copier l'environnement créé dans un fichier .py et on importe l'environnement dans `flow.envs.__init__.py`.
Mettre le chemin absolu du fichier .osm .
</font> 

In [6]:
# possibles actions
action_spec = OrderedDict({ "30677963": [ "GGGGrrrGGGG", "rrrrGGGrrrr"],
                            "30763263": ["GGGGGGGGGG",  "rrrrrrrrrr"],
                            "30677810": [ "GGrr", "rrGG"]})

In [7]:
horizon  = 1000
SIM_STEP = 0.1
n_veh    = 12
rollouts = 10
n_cpus   = 2
discount_rate = 0.999

In [8]:
# SUMO PARAM
sumo_params = SumoParams(sim_step=SIM_STEP, render=False, restart_instance=True)

# ENVIRONMENT PARAM
ADDITIONAL_ENV_PARAMS = {"beta": n_veh, "action_spec": action_spec, "algorithm": "DQN", "tl_constraint_min": 100,  "tl_constraint_max": 600, "sim_step": SIM_STEP}
env_params = EnvParams(additional_params=ADDITIONAL_ENV_PARAMS, horizon=horizon, warmup_steps=1)

# NETWORK PARAM
path_file  = '/home/lino/Documents/DQN_CIL4SYS/DQN_CIL4SYS/notebooks/issy.osm'
net_params = NetParams(inflows=inflow, osm_path=path_file) 

# NETWORK
network = IssyOSMNetwork

# INITIAL CONFIG
initial_config = InitialConfig(edges_distribution=EDGES_DISTRIBUTION)


flow_params = dict( exp_tag   = "ISSY_traffic", 
                    env_name  = IssyEnv1,  
                    network   = IssyOSMNetwork,
                    simulator = 'traci',
                    sim       = sumo_params,
                    env       = env_params,
                    net       = net_params,
                    veh       = vehicles,
                    initial   = initial_config)

# Setup RLlib library

Configures RLlib DQN algorithm to be used to train the RL model.

In [9]:
def setup_DQN_exp():

    alg_run   = 'DQN'
    agent_cls = get_agent_class(alg_run)
    config    = agent_cls._default_config.copy()
    config['num_workers']      = n_cpus
    config['train_batch_size'] = horizon * rollouts
    config['gamma']            = discount_rate
    config['clip_actions']     = False  # FIXME(ev) temporary ray bug
    config['horizon']          = horizon
    config["hiddens"]          = [256]
    config['model'].update({'fcnet_hiddens': [32, 32]})

    # save the flow params for replay
    flow_json = json.dumps(flow_params, cls=FlowParamsEncoder, sort_keys=True, indent=4)
    config['env_config']['flow_params'] = flow_json
    config['env_config']['run'] = alg_run

    create_env, gym_name = make_create_env(params=flow_params, version=0)

    # Register as rllib env
    register_env(gym_name, create_env)
    
    return alg_run, gym_name, config

Configures RLlib PPO algorithm to be used to train the RL model.

See: https://ray.readthedocs.io/en/latest/rllib-algorithms.html#proximal-policy-optimization-ppo

In [10]:
def setup_PPO_exp():

    alg_run   = 'PPO'
    agent_cls = get_agent_class(alg_run)
    config    = agent_cls._default_config.copy()
    config['num_workers']      = n_cpus
    config['train_batch_size'] = horizon * rollouts
    config['gamma']            = discount_rate
    config['use_gae']          = True
    config['lambda']           = 0.97
    config['kl_target']        = 0.02
    config['num_sgd_iter']     = 10
    config['clip_actions']     = False  # FIXME(ev) temporary ray bug
    config['horizon']          = horizon
    config['model'].update({'fcnet_hiddens': [32, 32]})

    # save the flow params for replay
    flow_json = json.dumps(flow_params,cls=FlowParamsEncoder,sort_keys=True,indent=4)
    config['env_config']['flow_params'] = flow_json
    config['env_config']['run'] = alg_run

    create_env, gym_name = make_create_env(params=flow_params,version=0)

    # Register as rllib env
    register_env(gym_name, create_env)
    
    return alg_run, gym_name, config

# Run Experiment

In [11]:
alg_run, gym_name, config = setup_DQN_exp()

ray.init(num_cpus=n_cpus + 1)

2020-04-14 10:51:05,266	INFO node.py:498 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2020-04-14_10-51-05_265940_6292/logs.
2020-04-14 10:51:05,461	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:44064 to respond...
2020-04-14 10:51:05,779	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:45022 to respond...
2020-04-14 10:51:05,784	INFO services.py:809 -- Starting Redis shard with 2.93 GB max memory.
2020-04-14 10:51:05,921	INFO node.py:512 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2020-04-14_10-51-05_265940_6292/logs.
2020-04-14 10:51:05,925	INFO services.py:1475 -- Starting the Plasma object store with 4.4 GB memory using /dev/shm.


{'node_ip_address': '192.168.0.48',
 'redis_address': '192.168.0.48:44064',
 'object_store_address': '/tmp/ray/session_2020-04-14_10-51-05_265940_6292/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-04-14_10-51-05_265940_6292/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2020-04-14_10-51-05_265940_6292'}

In [12]:
exp_tag = {"run": alg_run,
           "env": gym_name,
           "config": {**config},
           "checkpoint_freq": 2,
           "checkpoint_at_end": True,
           "max_failures": 999,
           "stop": {"training_iteration": 6}}


trials = run_experiments({flow_params["exp_tag"]: exp_tag})

2020-04-14 10:51:06,688	INFO trial_runner.py:176 -- Starting a new experiment.
2020-04-14 10:51:06,710	WARNING signature.py:108 -- The function with_updates has a **kwargs argument, which is currently not supported.
2020-04-14 10:51:06,744	ERROR log_sync.py:34 -- Log sync requires cluster to be setup with `ray up`.


== Status ==
Using FIFO scheduling algorithm.
Resources requested: 0/3 CPUs, 0/0 GPUs
Memory usage on this node: 3.0/14.7 GB



2020-04-14 10:51:06,912	WARNING util.py:145 -- The `start_trial` operation took 0.20603227615356445 seconds to complete, which may be a performance bottleneck.


== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/0 GPUs
Memory usage on this node: 3.0/14.7 GB
Result logdir: /home/lino/ray_results/ISSY_traffic
Number of trials: 1 ({'RUNNING': 1})
RUNNING trials:
 - DQN_IssyEnv1-v0_0:	RUNNING

(pid=6361) /home/lino/anaconda3/envs/flow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=6361)   _np_qint8 = np.dtype([("qint8", np.int8, 1)])
(pid=6361) /home/lino/anaconda3/envs/flow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=6361)   _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
(pid=6361) /home/lino/anaconda3/envs/flow/lib/python3.6/site-packages

(pid=6359) Warning: Discarding unknown compound 'cycleway.opposite' in type 'cycleway.opposite|highway.residential' (first occurence for edge '4786965#0').
(pid=6359) Warning: Discarding unknown compound 'cycleway.opposite_track' in type 'cycleway.opposite_track|highway.residential' (first occurence for edge '4786972#0').
(pid=6359) Warning: Discarding unknown compound 'cycleway.opposite' in type 'cycleway.opposite|highway.living_street' (first occurence for edge '4795758').
(pid=6359) Warning: Discarding unknown compound 'cycleway.lane' in type 'cycleway.lane|highway.secondary' (first occurence for edge '100822066').
(pid=6360) Warning: Discarding unknown compound 'cycleway.opposite' in type 'cycleway.opposite|highway.residential' (first occurence for edge '4786965#0').
(pid=6360) Warning: Discarding unknown compound 'cycleway.opposite_track' in type 'cycleway.opposite_track|highway.residential' (first occurence for edge '4786972#0').
(pid=6360) Warning: Discarding unknown compound 'c

(pid=6360) 2020-04-14 10:51:20,400	INFO sampler.py:304 -- Raw obs from env: { 0: { 'agent0': np.ndarray((112,), dtype=float64, min=0.0, max=2637.506, mean=292.907)}}
(pid=6360) 2020-04-14 10:51:20,400	INFO sampler.py:305 -- Info return from env: {0: {'agent0': None}}
(pid=6360) 2020-04-14 10:51:20,401	INFO sampler.py:403 -- Preprocessed obs: np.ndarray((112,), dtype=float64, min=0.0, max=2637.506, mean=292.907)
(pid=6360) 2020-04-14 10:51:20,401	INFO sampler.py:407 -- Filtered obs: np.ndarray((112,), dtype=float64, min=0.0, max=2637.506, mean=292.907)
(pid=6360) 2020-04-14 10:51:20,402	INFO sampler.py:521 -- Inputs to compute_actions():
(pid=6360) 
(pid=6360) { 'default_policy': [ { 'data': { 'agent_id': 'agent0',
(pid=6360)                                   'env_id': 0,
(pid=6360)                                   'info': None,
(pid=6360)                                   'obs': np.ndarray((112,), dtype=float64, min=0.0, max=2637.506, mean=292.907),
(pid=6360)                         

(pid=6360) Warning: Vehicle 'flow_20.10' performs emergency braking with decel=-63.94 wished=4.50 severity=13.21, time=30.20.
(pid=6360) Warning: Vehicle 'flow_20.9' performs emergency braking with decel=-140.76 wished=4.50 severity=30.28, time=30.20.
(pid=6360) Warning: Vehicle 'flow_20.11' performs emergency braking with decel=-9.61 wished=4.50 severity=1.14, time=30.30.
(pid=6359) Warning: Choosing new speed factor 1.26 for vehicle 'flow_40.8' to match departure speed.
(pid=6359) Warning: Choosing new speed factor 1.26 for vehicle 'flow_40.9' to match departure speed.
(pid=6360) Warning: Choosing new speed factor 1.26 for vehicle 'flow_40.6' to match departure speed.
(pid=6359) Warning: Vehicle 'flow_40.0' performs emergency braking with decel=-11.68 wished=4.50 severity=1.60, time=38.70.
(pid=6359) Warning: Choosing new speed factor 1.26 for vehicle 'flow_40.10' to match departure speed.
(pid=6359) Warning: Choosing new speed factor 1.26 for vehicle 'flow_40.11' to match departure 

2020-04-14 10:51:40,705	ERROR worker.py:1716 -- listen_error_messages_raylet: Error 111 connecting to 192.168.0.48:44064. Connection refused.
2020-04-14 10:51:40,706	ERROR import_thread.py:89 -- ImportThread: Error 111 connecting to 192.168.0.48:44064. Connection refused.
2020-04-14 10:51:40,706	ERROR worker.py:1616 -- print_logs: Error 111 connecting to 192.168.0.48:44064. Connection refused.


KeyboardInterrupt: 